# Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

'2.2.0'

# Data preprocessing

## Importing the dataset

In [ ]:
df = pd.read_csv('Churn_Modelling.csv')

In [ ]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
X = df.iloc[:, 3:-1].values
y = df.iloc[:, -1].values

## Encoding categorical data

### Label encoding the gender column

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
pd.DataFrame(X).head()

,0,1,2,3,4,5,6,7,8,9
0,619,France,0,42,2,0,1,1,1,101349
1,608,Spain,0,41,1,83807.9,1,0,1,112543
2,502,France,0,42,8,159661,3,1,0,113932
3,699,France,0,39,1,0,2,0,0,93826.6
4,850,Spain,0,43,2,125511,1,1,1,79084.1


### One hot encoding the geography column

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
pd.DataFrame(X).head()  

,0,1,2,3,4,5,6,7,8,9,10,11
0,1,0,0,619,0,42,2,0,1,1,1,101349
1,0,0,1,608,0,41,1,83807.9,1,0,1,112543
2,1,0,0,502,0,42,8,159661,3,1,0,113932
3,1,0,0,699,0,39,1,0,2,0,0,93826.6
4,0,0,1,850,0,43,2,125511,1,1,1,79084.1


## Splitting the dataset into training set and test set


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size = 0.2, random_state = 0)

## Feature scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Creating the ANN

## Initialising the ANN

In [ ]:
ann = tf.keras.models.Sequential()

## Adding input layer and first hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

## Adding the second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

## Adding output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

## Training the ANN

### Compiling the ANN

In [ ]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on training set

In [ ]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5661 - accuracy: 0.7775
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4776 - accuracy: 0.7995
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4480 - accuracy: 0.8110
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4348 - accuracy: 0.8142
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4262 - accuracy: 0.8184
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4193 - accuracy: 0.8215
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4130 - accuracy: 0.8214
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4068 - accuracy: 0.8246
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4007 - accuracy: 0.8309
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3959 - accura

# Making predictions

## Predicting single result

In [ ]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))

[[0.0333349]]


## Predicting test set results

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


# Analysing the results

## Making the confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
print("Confusion matrix: 'n{0} \nAccuracy_score: {1}".format(cm, acc))

Confusion matrix: 'n[[1510   85]
 [ 197  208]] 
Accuracy_score: 0.859
